In [1225]:
import pandas as pd 



In [1226]:
df = pd.read_csv('aug_train.csv')
df.head()


,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


In [1227]:
df.info()
df.shape


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevent_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  target                  19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

(19158, 14)

In [1228]:
for column in df[['gender', 'relevent_experience', 'enrolled_university', 'education_level']]:
    print(f"{column} nans are: {df[column].isna().sum()}")

gender nans are: 4508
relevent_experience nans are: 0
enrolled_university nans are: 386
education_level nans are: 460


In [1229]:
for column in df.iloc[:, 3:]:
    print(f"{column} values are: {df[column].value_counts()}\n {column} nans are: {df[column].isna().sum()}\n ----")

gender values are: gender
Male      13221
Female     1238
Other       191
Name: count, dtype: int64
 gender nans are: 4508
 ----
relevent_experience values are: relevent_experience
Has relevent experience    13792
No relevent experience      5366
Name: count, dtype: int64
 relevent_experience nans are: 0
 ----
enrolled_university values are: enrolled_university
no_enrollment       13817
Full time course     3757
Part time course     1198
Name: count, dtype: int64
 enrolled_university nans are: 386
 ----
education_level values are: education_level
Graduate          11598
Masters            4361
High School        2017
Phd                 414
Primary School      308
Name: count, dtype: int64
 education_level nans are: 460
 ----
major_discipline values are: major_discipline
STEM               14492
Humanities           669
Other                381
Business Degree      327
Arts                 253
No Major             223
Name: count, dtype: int64
 major_discipline nans are: 2813
 ----
exp

In [1230]:


df.loc[:, 'gender'] = df.gender.fillna('none')
df.gender.isna().sum()


0

In [1231]:
# df.enrolled_university.value_counts()
df[df.enrolled_university.isna()]
df.enrolled_university.isna().sum()

386

In [1232]:
condition = (df.enrolled_university.isna() & df.major_discipline.isna() & df.education_level.isna())

condition_1 = df[df.enrolled_university.isna() & ((df.education_level == 'High School') | (df.education_level == 'Primary School'))]
df = df[~ condition ]  
df.enrolled_university.isna().sum()


286

In [1233]:
# df['enrolled_university'] = df[df.enrolled_university.isna() & ((df.education_level == 'High School') | (df.education_level == 'Primary School'))]
df.loc[(df.enrolled_university.isna()) & ((df.education_level == 'High School') | (df.education_level == 'Primary School')), 'enrolled_university' ] = 'no_enrollment'




In [1234]:
df = df.dropna(subset=['enrolled_university'])
df = df.dropna(subset=['education_level'])
condition_2 = df[(df.major_discipline.isna()) & ((df.education_level == "High School") | (df.education_level == 'Primary School'))]
df.loc[(df.major_discipline.isna()) & ((df.education_level == "High School") | (df.education_level == 'Primary School')), 'major_discipline'] = 'No Major'

In [1235]:
df.major_discipline.isna().sum()

# df[df.major_discipline.isna() & ((df.education_level == 'Masters') | (df.education_level == 'Phd') | (df.education_level == 'Graduate'))]

27

In [1236]:
# df[df.major_discipline.isna()]

df.major_discipline.value_counts()


major_discipline
STEM               14284
No Major            2544
Humanities           660
Other                377
Business Degree      325
Arts                 251
Name: count, dtype: int64

In [1237]:

df = df.drop(df[(df.major_discipline.isna()) & ((df.education_level == 'Masters') | (df.education_level == 'Phd') | (df.education_level == 'Graduate'))].index)
df[df.major_discipline.isna() & ((df.education_level == 'Masters') | (df.education_level == 'Phd') | (df.education_level == 'Graduate'))]

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target


In [1238]:
condition_3 = (df['major_discipline'] == 'No Major') & ~((df['education_level'] == 'Primary School') | (df['education_level'] == 'High School'))
df = df.drop(df[condition_3].index)
df.experience.isna().sum()

51

In [1239]:
df['experience'] = df.experience.str.replace('<', '').str.replace('>', '')
df['experience'] = df['experience'].astype(float)


df['experience'] = df.experience.isna() == df.experience.mean()


In [1242]:
df.company_type.value_counts()

company_type
Pvt Ltd                9487
Funded Startup          981
Public Sector           925
Early Stage Startup     570
NGO                     501
Other                   116
Name: count, dtype: int64